In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from datasets import load_dataset

# loading the dataset
ds = load_dataset('food101')

# getting an example
ex = ds['train'][400]
print(ex)

# seeing the image
image = ex['image']
image.show()

# getting all the labels
labels = ds['train'].features['label']
print(labels)

# getting label of our example
print(labels.int2str(ex['label']))

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=384x512 at 0x27459A1C820>, 'label': 6}
ClassLabel(names=['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 

In [3]:
from transformers import AutoFeatureExtractor

#loading the feature extractor
model_name= 'microsoft/swin-base-patch4-window7-224'
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert('RGB') for x in example_batch['image']], return_tensors='pt')
    inputs['label'] = example_batch['label']
    return inputs
  
# applying transform
prepared_ds = ds.with_transform(transform)

In [4]:
import torch

def collate_fn(batch):
  #data collator
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [5]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
  # function which calculates accuracy for a certain set of predictions
  return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-5-6effe46d2a76>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [6]:
from transformers import SwinForImageClassification, Trainer, TrainingArguments

labels = ds['train'].features['label'].names

# initialzing the model
model = SwinForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes = True,
)

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([101, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([101]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
len(labels)

101

In [9]:
from transformers import Trainer, TrainingArguments

batch_size = 16
# Defining training arguments (set push_to_hub to false if you don't want to upload it to HuggingFace's model hub)
training_args = TrainingArguments(
    f"swin-finetuned-food101",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [10]:
# Instantiate the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

In [11]:
# Train and save results
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

# Evaluate on validation set
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

C:\Users\Sang\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 75750
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 3549
  Number of trainable parameters = 86846749


Epoch,Training Loss,Validation Loss,Accuracy
0,0.528400,0.381670,0.886970
1,0.364600,0.304511,0.910970
2,0.144400,0.274853,0.923406


***** Running Evaluation *****
  Num examples = 25250
  Batch size = 16
Saving model checkpoint to swin-finetuned-food101\checkpoint-1183
Configuration saved in swin-finetuned-food101\checkpoint-1183\config.json
Model weights saved in swin-finetuned-food101\checkpoint-1183\pytorch_model.bin
Image processor saved in swin-finetuned-food101\checkpoint-1183\preprocessor_config.json
***** Running Evaluation *****
  Num examples = 25250
  Batch size = 16
Saving model checkpoint to swin-finetuned-food101\checkpoint-2366
Configuration saved in swin-finetuned-food101\checkpoint-2366\config.json
Model weights saved in swin-finetuned-food101\checkpoint-2366\pytorch_model.bin
Image processor saved in swin-finetuned-food101\checkpoint-2366\preprocessor_config.json
***** Running Evaluation *****
  Num examples = 25250
  Batch size = 16
Saving model checkpoint to swin-finetuned-food101\checkpoint-3549
Configuration saved in swin-finetuned-food101\checkpoint-3549\config.json
Model weights saved in swi

***** train metrics *****
  epoch                    =           3.0
  total_flos               = 16597913700GF
  train_loss               =        0.6267
  train_runtime            =   15:54:16.67
  train_samples_per_second =         3.969
  train_steps_per_second   =         0.062


***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.9234
  eval_loss               =     0.2749
  eval_runtime            = 0:08:39.14
  eval_samples_per_second =     48.638
  eval_steps_per_second   =      3.042
